# TACO Experiment: Cherry-picked context to one specific task

The goal here is to evaluate how the model will behave when passing the Solutions from an analog problem that were manually analyzed and selected  
In this specific scenario we want to test it with a very specific group of tasks from the TACO benchmark

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import polars as pl
import torch
import numpy as np
import json

## LLM
from src.llms import Llama3_1_Instruct

seed = 42
# NumPy
np.random.seed(seed)

# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of GPUs available: 1
GPU 0: NVIDIA RTX A5000


## Load Datasets

In [3]:
PATH  = "../../../data/TACO/processed"
train = pl.read_ipc(f"{PATH}/train.feather")
train_solutions = pl.read_ipc(f"{PATH}/train_solutions.feather")

In [ ]:
def run_inference(prompt_input: str, path: str):
    outputs = []
    llm = Llama3_1_Instruct()
    config = {
                "temperature": 0.7,
                "max_length": 2048,
                "top_p": 0.95,
                "num_return_sequences": 20
    }

    prompt = f"Please write a Python program \nQUESTION: \n{prompt_input} \n ANSWER: \n."
    output = llm.run(prompt=prompt, input="", config_params=config)

    for res in output:
        outputs.append(res)[]
    
    json.dump(outputs, open(path, "w"))

## Problem Selection

The category choosen it was "Geometry" in EASY difficulty  
The criteria behind the choice is because there isn't a lot of examples of geometry, which facilitates to find samples specific from that scope.  
The EASY difficulty is for validation purposes

In [8]:
train.filter(pl.col("tags") == "Geometry").filter(pl.col("difficulty") == "EASY").count()

id,difficulty,tags,input
u32,u32,u32,u32
127,127,127,127


In [9]:
selected_problem = train.filter(pl.col("tags") == "Geometry").filter(pl.col("difficulty") == "EASY").sample(1)
print(selected_problem)

shape: (1, 4)
┌───────┬────────────┬──────────┬─────────────────────────────────┐
│ id    ┆ difficulty ┆ tags     ┆ input                           │
│ ---   ┆ ---        ┆ ---      ┆ ---                             │
│ u32   ┆ str        ┆ str      ┆ str                             │
╞═══════╪════════════╪══════════╪═════════════════════════════════╡
│ 23834 ┆ EASY       ┆ Geometry ┆ Johnny needs to make a rectang… │
└───────┴────────────┴──────────┴─────────────────────────────────┘


In [16]:
print(selected_problem.select("input").to_struct())

shape: (1,)
Series: '' [struct[1]]
[
	{"Johnny needs to make a rectangular box for his physics class project. He has bought P cm of wire and S cm^{2} of special paper. He would like to use all the wire (for the 12 edges) and paper (for the 6 sides) to make the box.
What is the largest volume of the box that Johnny can make?

------ Input ------ 

The first line contains t, the number of test cases (about 10). Then t test cases follow.

Each test case contains two integers P and S in a line (1 ≤ P ≤ 40000, 1 ≤ S ≤ 20000). You may assume that there always exists an optimal solution for the given input cases.

------ Output ------ 

For each test case, print a real number that is the largest volume of the box that Johnny can make, rounded to two decimal places.

----- Sample Input 1 ------ 
2
20 14
20 16
----- Sample Output 1 ------ 
3.00
4.15
----- explanation 1 ------ 
First case: the dimensions of the largest box may be 3, 1 and 1.
Second case: the dimensions of the largest box may be 

## Run Baseline - No Context

In [ ]:
run_inference(selected_problem.select("input").to_struct().to_pandas().iloc[0]["input"], "no_context.json")